In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, min, substring_index, avg, expr
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import os, csv
from pmdarima import auto_arima
# from statsmodels.tsa.holtwinters import ExponentialSmoothing
# from sklearn.metrics import mean_squared_error
# import numpy as np

In [2]:
#Create Spark Session
spark = SparkSession.builder.appName("FlubyRegion").getOrCreate()

In [3]:
root_dir = "flu_region"
data_dir = "data"

In [4]:
#Divide Flu-by-Region into Flu-by-County
filecheck1 = f"{root_dir}/flu_by_county.csv"
if not os.path.isfile(filecheck1):
    fbc_dict = {
    "Bay Area": ["01675839" , "01675903", "00277285", "00277292", "00277302", "00277305" , "00277307",  "00277308" , "00277312", "01657246"],
    "Central": ["01675885", "00277274", "01804637", "00277280", "00277284", "00277286", "00277288", "00277290", "00277291", "00277299", "00277303", "00277314", "00277318", "00277319"],
    "Lower Southern" : ["00277277", "00277294", "00277297", "00277300", "00277301"],
    "Northern" : ["01675840", "01675841", "01675842", "01675902", "01682074", "00277273", "00277275", "01681908", "00277281", "01693324", "00277287", "00277289", "01682927", "00277295", "00277296",  "00277298", "01682610", "00277310", "00277311", "00277315", "01692767", "00277317", "00277321", "00277322"],
    "Upper Southern" : ["02054176", "00277283", "00277304", "00277306", "00277320"]
    }

    df = pd.read_csv(f"{data_dir}/flu-ili-byregion-fluseason.csv")
    # Create an empty list to store new dataframes
    new_dfs = []

    # Loop through each row in the original DataFrame
    for _, row in df.iterrows():
        x_value = row["region"]
        values_to_replace = fbc_dict.get(x_value, [x_value])

        # Create new dataframes for each value in values_to_replace
        for new_x_value in values_to_replace:
            new_df = row.copy().to_frame().T  # Create a new DataFrame with one row
            new_df["region"] = new_x_value
            new_dfs.append(new_df)

    # Concatenate the list of dataframes into a single dataframe
    new_df = pd.concat(new_dfs, ignore_index=True)

    # Write the modified DataFrame to a CSV file
    new_df.to_csv(f"{root_dir}/flu_by_county.csv", index=False)

df = spark.read.csv(f"{root_dir}/flu_by_county.csv", header=True, inferSchema=True)
df.show()

+---------+---------+----------+----------+---------+-------------------+-----------+--------------------------+
|   season|date_code|weekending|    region|Total_ILI|Total_Patients_Seen|Percent_ILI|Number_Providers_Reporting|
+---------+---------+----------+----------+---------+-------------------+-----------+--------------------------+
|2001-2002|   200140| 10/6/2001|  01675839|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  01675903|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  00277285|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  00277292|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  00277302|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  00277305|        3|                135|       2.22|           

In [7]:
#Divide data into separate files by date
filecheck2 = f"{root_dir}/flu_by_dates"
if not os.listdir(filecheck2):
    # Create a Spark session

    # Read your data into a DataFrame (replace "input_file.csv" with your file)
    df = spark.read.csv(f"{root_dir}/flu_by_county.csv", header=True, inferSchema=True)

    # Define the column to group by
    group_column = "date_code"  # Replace with your actual column name

    # Get distinct values in the group column
    distinct_values = [str(row[group_column]) for row in df.select(group_column).distinct().collect()]

    # Group by the column and write each group to a separate file
    for value in distinct_values:
        group_df = df.filter(col(group_column) == value)
        output_path = f"{filecheck2}/{value}.csv"  # Replace with your desired output directory
        pandas_df = group_df.toPandas()
        pandas_df.to_csv(output_path, header=True, index=False)


In [13]:
# Generate Heatmaps
maps_dir = f"{root_dir}/maps"

# Check if the directory is empty
if not os.listdir(maps_dir):
    ca_map = gpd.read_file(f"data/CA_Counties/CA_Counties_TIGER2016.shp")
    
    # List all csv files
    csv_files = [filename for filename in os.listdir(filecheck2) if filename.endswith(".csv")]

    # Iterate through the subdirectories and read CSV files in each subdirectory
    for csv_file in csv_files:
        
        csv_file_path = os.path.join(filecheck2, csv_file)
        df = pd.read_csv(csv_file_path)

        # Calculate the maximum and minimum values in the "Total_ILI" column
        max_value = df["Total_ILI"].max()
        min_value = df["Total_ILI"].min()

        #Merge files
        mapf = "COUNTYNS"
        statf = "region"
        map_and_stats = ca_map.merge(df, left_on=mapf, right_on=statf)

        #plot map
        fig, ax = plt.subplots(1, figsize=(8, 8))
        plt.xticks(rotation=90)
        map_and_stats.plot(column="Total_ILI", cmap="Reds", linewidth=0.4, ax=ax, edgecolor=".4")
        title_column = "weekending"
        title_value = df[title_column].iloc[0]
        plt.title(f"Total IRI for Week of {title_value}")
        bar_info = plt.cm.ScalarMappable(cmap="Reds", norm=plt.Normalize(vmin=min_value, vmax=max_value))
        bar_info._A = []
        cbar = fig.colorbar(bar_info)

        # Save the map as an image file in the "maps" directory
        output_file = os.path.join(maps_dir, os.path.splitext(csv_file)[0] + ".png")
        plt.savefig(output_file, dpi=300)  # Change the dpi and format as needed

        # Close the figure to release memory
        plt.close(fig)

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the A

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the A

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the A

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the A

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the A

...

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the A

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the A

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the A

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)
C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the A

C:\Users\jourd\AppData\Local\Temp\ipykernel_23880\3706447613.py:35: MatplotlibDeprecationWarning: Unable to determine Axes to steal space for Colorbar. Using gca(), but will raise in the future. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.
  cbar = fig.colorbar(bar_info)


In [9]:
# Load your time series data into a PySpark DataFrame
df = spark.read.csv(f"{root_dir}/flu_by_county.csv", header=True, inferSchema=True)
df = df.filter(df["region"] != "California")

df = df.withColumn("weekending", expr("substring_index(weekending, '/', 1) || '/' || substring_index(weekending, '/', -1)"))

# Group by season, month, and region using expr
grouped_df = df.groupBy(
    expr("season"),
    expr("weekending").alias("month"),
    col("region")
).agg(avg(col("Total_ILI")).alias("avg_total_ili"))

# Order by season, month, and region
grouped_df = grouped_df.orderBy("season", "month", "region")

# Show the DataFrame
grouped_df.show()

grouped_df= grouped_df.toPandas()
grouped_df.to_csv(f"{root_dir}/avg_flu_by_county.csv", index=False)


+---------+------+--------+-------------+
|   season| month|  region|avg_total_ili|
+---------+------+--------+-------------+
|2001-2002|1/2002|00277273|         24.5|
|2001-2002|1/2002|00277274|         29.0|
|2001-2002|1/2002|00277275|         24.5|
|2001-2002|1/2002|00277277|          1.5|
|2001-2002|1/2002|00277280|         29.0|
|2001-2002|1/2002|00277281|         24.5|
|2001-2002|1/2002|00277283|        17.25|
|2001-2002|1/2002|00277284|         29.0|
|2001-2002|1/2002|00277285|         46.0|
|2001-2002|1/2002|00277286|         29.0|
|2001-2002|1/2002|00277287|         24.5|
|2001-2002|1/2002|00277288|         29.0|
|2001-2002|1/2002|00277289|         24.5|
|2001-2002|1/2002|00277290|         29.0|
|2001-2002|1/2002|00277291|         29.0|
|2001-2002|1/2002|00277292|         46.0|
|2001-2002|1/2002|00277294|          1.5|
|2001-2002|1/2002|00277295|         24.5|
|2001-2002|1/2002|00277296|         24.5|
|2001-2002|1/2002|00277297|          1.5|
+---------+------+--------+-------

In [ ]:
#Predictive map
filecheck3 = f"{root_dir}/avg_flu_by_county.csv"
if os.path.isfile(filecheck3):
    # Read the CSV file into a PySpark DataFrame
    df = spark.read.csv(f"{root_dir}/avg_flu_by_county.csv", header=True, inferSchema=True)
    #df.show()
    # Select the relevant columns
    df = df.select("month", "region", "avg_total_ili")

    # Convert the PySpark DataFrame to a Pandas DataFrame
    pandas_df = df.toPandas()

    # Convert the "weekending" column to a datetime format
    pandas_df["month"] = pd.to_datetime(pandas_df["month"])

    # Create the Matplotlib plot
    plt.plot(pandas_df["month"], pandas_df["avg_total_ili"])

    # Set labels for the x and y axes
    plt.xlabel("Date")
    plt.ylabel("Average Total_ILI")
    
    df = df.select("month", "avg_total_ili")
    df = df.withColumnRenamed("avg_total_ili", "target")

    # Collect the data to the driver for processing with Pyramid ARIMA
    data = df.select("month", "target").orderBy("month").toPandas()

    # Train the ARIMA model using the Pyramid ARIMA library
    # You can adjust the ARIMA parameters as needed
    stepwise_model = auto_arima(
        data["target"],
        seasonal=True,
        m=12,  # Define the seasonality (e.g., 12 for monthly data)
        stepwise=True
    )

    # Make predictions
    # You can replace this with your actual future data
    future_data = df.select("month").distinct().toPandas()
    predictions, conf_int = stepwise_model.predict(n_periods=future_data.shape[0], return_conf_int=True)

    # Combine predictions with future data
    future_data["predicted_ILI"] = predictions

    # Display or save the predictions
    # You can save the predictions as needed or show them
    future_data.show()


    # Show the plot
    plt.show()

In [ ]:
spark.stop()

In [39]:
df = spark.read.csv(f"{root_dir}/flu_by_county.csv", header=True, inferSchema=True)
df = df.filter(df["region"] != "California")
df = df.withColumn("weekending", expr("substring_index(weekending, '/', 1) || '/' || substring_index(weekending, '/', -1)"))
df.show()
# s = "10/11/1222"
# first = s.find("/")
# s_end = s[first+1:]
# secnd = s_end.find("/")
# s_new = s[:first] + s_end[secnd:]
# print(s_new)

+---------+---------+----------+--------+---------+-------------------+-----------+--------------------------+
|   season|date_code|weekending|  region|Total_ILI|Total_Patients_Seen|Percent_ILI|Number_Providers_Reporting|
+---------+---------+----------+--------+---------+-------------------+-----------+--------------------------+
|2001-2002|   200140|   10/2001|01675839|        3|                135|       2.22|                         2|
|2001-2002|   200140|   10/2001|01675903|        3|                135|       2.22|                         2|
|2001-2002|   200140|   10/2001|00277285|        3|                135|       2.22|                         2|
|2001-2002|   200140|   10/2001|00277292|        3|                135|       2.22|                         2|
|2001-2002|   200140|   10/2001|00277302|        3|                135|       2.22|                         2|
|2001-2002|   200140|   10/2001|00277305|        3|                135|       2.22|                         2|
|

In [45]:
# Create a DataFrame with a column for the month and another for the year
data = {
    'Month': [1, 2, 3, 4, 5],
    'Year': [2022, 2022, 2022, 2022, 2022]
}

df = pd.DataFrame(data)

# Create a datetime column with just month and year
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1))

# Print the DataFrame
print(df)

   Month  Year       Date
0      1  2022 2022-01-01
1      2  2022 2022-02-01
2      3  2022 2022-03-01
3      4  2022 2022-04-01
4      5  2022 2022-05-01


In [24]:
df = spark.read.csv(f"avg_flu_by_county.csv", header=True, inferSchema=True)
df = df.orderBy("season", "region")
df.show()
ddf = spark.read.csv(f"{root_dir}/flu_by_county.csv", header=True, inferSchema=True)
ddf = ddf.filter(ddf["region"] != "California")
ddf = ddf.groupBy("season", substring_index(col("weekending"), "/", 1).alias("month"), "region")

+---------+-----+------+--------------+
|   season|month|region|avg(Total_ILI)|
+---------+-----+------+--------------+
|2001-2002|    3|277273|          25.0|
|2001-2002|    8|277273|           0.0|
|2001-2002|    1|277273|          24.5|
|2001-2002|   12|277273|          19.4|
|2001-2002|    2|277273|          36.0|
|2001-2002|   10|277273|         25.25|
|2001-2002|   11|277273|          21.0|
|2001-2002|    4|277273|           7.0|
|2001-2002|    5|277273|          15.0|
|2001-2002|    6|277273|           1.6|
|2001-2002|    7|277273|          0.75|
|2001-2002|    9|277273|          1.75|
|2001-2002|    4|277274|           3.5|
|2001-2002|   10|277274|           0.0|
|2001-2002|    2|277274|          25.5|
|2001-2002|   11|277274|          11.0|
|2001-2002|    3|277274|          19.6|
|2001-2002|    5|277274|          0.25|
|2001-2002|    1|277274|          29.0|
|2001-2002|    6|277274|           0.0|
+---------+-----+------+--------------+
only showing top 20 rows

